<font size=6 color = "#663399" >Genetic Algorithms</font>
- <font size=4>Name: Kasra Noorbakhsh</font>
- <font size=4>SID: 810100230</font>
- <font size=4>In this project we trained on Genetic algorithms with Fractional Knapsack problem.</font>

<font size ="5" color ="#9400D3">Part 1: Basic Concepts</font>
- each value and weight of a food is considered a gene.
- A chromosome consists of genes with size : `POPULATION_SIZE`.

In [1]:
import random
import csv
import copy

In [2]:
CSV_PATH = "snacks.csv"

CROSSOVER_PROBABILITY = 0.7
CARRY_PROBABILITY = 0.1
MUTATION_PROBABILITY = 0.05

POPULATION_SIZE = 500
MAX_WEIGHT = 18
MIN_VALUE = 27
MIN_NUMBER_OF_FOODS = 1
MAX_NUMBER_OF_FOODS = 9
MAX_NUMBER_OF_GENERATION = 1000

<font size ="5" color ="#9400D3">Part 2: Basic Population</font>
- 

In [3]:
def make_first_population(population_size, chromosome_size): 

    first_population = []
    
    for i in range(population_size):
        chromosome = []
        for j in range(chromosome_size):
            chromosome.append(random.random()) 
        first_population.append(chromosome)
    
    return first_population

<font size ="5" color ="#9400D3">Part 3: Fitness</font>
- 

In [4]:
def calculate_properties_chromosome(chromosome, values, weights):
    total_weight = 0
    total_value = 0

    for i in range(len(chromosome)):
        total_weight += chromosome[i] * weights[i]
        total_value += chromosome[i] * values[i]

    return total_weight, total_value

def calculate_weights_values(chromosomes, population_size, Values, Weights):

    totalWeights = []
    totalValues = []
    indexes_of_foods = []

    for i in range(population_size):

        total_wei = 0
        total_val = 0

        size_of_chromosome = random.randint(MIN_NUMBER_OF_FOODS, MAX_NUMBER_OF_FOODS)
        random_food_indexs = random.sample(range(len(Weights)), size_of_chromosome)
        indexes_of_foods.append(random_food_indexs)

        for j in range(size_of_chromosome):
            total_wei += chromosomes[i][random_food_indexs[j]] * Weights[random_food_indexs[j]]
            total_val += chromosomes[i][random_food_indexs[j]] * Values[random_food_indexs[j]]
        totalWeights.append(total_wei)
        totalValues.append(total_val)

    for l in range(population_size):
        for w in range(len(Weights)):
            if(w not in indexes_of_foods[l]):
                chromosomes[l][w] = 0


    return totalWeights, totalValues, indexes_of_foods

def calculate_fitness(totalWeights, totalValues):

    initial_value = 0
    fittneses = [initial_value] * len(totalWeights)

    for k in range(len(totalWeights)):
        if(totalWeights[k] > 18 or totalValues[k] < 20) :
            fittneses[k] = 0
        else:
            fittneses[k] = totalValues[k]

    return fittneses

<font size ="5" color ="#9400D3">Part 4: Crossover & Mutation & Pool</font>


- <font size ="3" color ="#9400D3"> Pool: </font>
- Here I choose the wheel method to form the pool 

In [5]:
def create_mating_pool_wheel(population, fitnesses):

    list_len = len(population)
    sumOfFitness = sum(fitnesses)
    if(sumOfFitness == 0):
        matingPool = random.choices(population, k=POPULATION_SIZE)
        
    else:
        probList = [(fitnesses[i]/sumOfFitness) for i in range(0,list_len)]
        matingPool = random.choices(population,weights=probList,k=POPULATION_SIZE)

    random.shuffle(matingPool)
    return matingPool

- <font size ="3" color ="#9400D3"> Crossover: </font>
- I have 2 kinds of crossover: N-point and 1-point

In [6]:
def n_point_crossover(n, chromosome1, chromosome2):

    crossover_points = []
    swap_or_not = True

    for i in range(n):
        crossover_points.append(random.randint(0, len(chromosome1) - 1))

    crossover_points.sort()
    
    for j in range(len(crossover_points)):
        
        if(swap_or_not):
            if(j == len(crossover_points) - 1):
                chromosome1[crossover_points[j]:], chromosome2[crossover_points[j]:] = chromosome2[crossover_points[j]:], chromosome1[crossover_points[j]:]
            else:
                chromosome1[crossover_points[j]:crossover_points[j + 1]], chromosome2[crossover_points[j]:crossover_points[j + 1]] = chromosome2[crossover_points[j]:crossover_points[j + 1]], chromosome1[crossover_points[j]:crossover_points[j + 1]]
            swap_or_not = False
        else:
            swap_or_not = True
            
def crossover_on_list(population, values, weights):
    population_len = len(population)

    for i in range(0,population_len,2):

        if i + 1 == population_len:
            break
        
        if(random.random() < CROSSOVER_PROBABILITY):
            ch1 = copy.deepcopy(population[i])
            ch2 = copy.deepcopy(population[i+1])
            
            n_point_crossover(3, ch1, ch2)

            total_weight1, total_value1 = calculate_properties_chromosome(ch1, values, weights)
            total_weight2, total_value2 = calculate_properties_chromosome(ch2, values, weights)

            if(not (total_weight1 > MAX_WEIGHT or total_value1 < MIN_VALUE)):
                population[i] = ch1
            if(not (total_weight2 > MAX_WEIGHT or total_value2 < MIN_VALUE)):
                population[i+1] = ch2

- <font size ="3" color ="#9400D3"> Mutation: </font>
- some of the chromosomes do not mutate at all and are forwarded

In [7]:
def mutate(chromosome):
    for i in range(len(chromosome)):
        if(not isinstance(chromosome[i], int)):
            if(random.random() < CARRY_PROBABILITY):
                chromosome[i] = random.random()

def mutate_on_list(population, values, weights):

    populationLen = len(population)

    for i in range(populationLen):
        if(random.random() < MUTATION_PROBABILITY):
            ch = copy.deepcopy(population[i])
            mutate(ch)
            
            total_weight, total_value = calculate_properties_chromosome(ch, values, weights)

            if(not (total_weight > MAX_WEIGHT or total_value < MIN_VALUE)):
                population[i] = ch

<font size ="5" color ="#9400D3">Part 5: Algorithm</font>
- 

In [8]:
Foods = []
Weights = []
Values = []

file = open(CSV_PATH)
csvreader = csv.reader(file)

for row in csvreader:
    Foods.append(row)

for i in range(1,len(Foods)):
    Weights.append(int(Foods[i][1]))
    Values.append(int(Foods[i][2]))
    
def find_sol1(population_size):
    
    population = make_first_population(population_size, len(Weights))

    best_answer = []
    bestAnswerFitness = 0

    for i in range(MAX_NUMBER_OF_GENERATION):

        total_weights, total_values, indexes = calculate_weights_values(population, population_size, Values, Weights)

        fitnesha = calculate_fitness(total_weights, total_values)

        sorted_list = [x for _, x in sorted(zip(fitnesha, population))]

        newBestAnswer = sorted_list[0]
        newBestAnswerFitness = max(fitnesha)

        if bestAnswerFitness < newBestAnswerFitness:   
            bestAnswerFitness = newBestAnswerFitness
            best_answer = newBestAnswer
        
        population = create_mating_pool_wheel(population, fitnesha)
        crossover_on_list(population, Values, Weights)
        mutate_on_list(population, Values, Weights)

    return best_answer, bestAnswerFitness

best_answer, bestAnswerFitness = find_sol1(POPULATION_SIZE)

<font size ="5" color ="#9400D3">Part 6: Results</font>
- 

In [9]:
total_wei = 0

for i in range(len(best_answer)):
    if(best_answer[i] != 0):
        total_wei += best_answer[i] * int(Foods[i][1])
        print(Foods[i][0], ":", best_answer[i] * int(Foods[i][1]))

print("Total Weight:", total_wei)
print("Total Value:", bestAnswerFitness)

Saghe-Talaei : 0.25013420848035717
Total Weight: 0.25013420848035717
Total Value: 24.850119691384897


<font size ="5" color ="#9400D3">Part 6: Questions</font>
- If we choose a very small population then we may not consider some of answers for example if we have 10 genes in a chromosome and first population of size 10 then we miss the 2^10 - 10 probebilities. If we have a large population then in one of our randoms we have the correct answer and runtime and memory usage increases.

- By growing the population size, the diversity increases and it may help us but the resources(time and memory) consumption will increase. Also we keep the population the same to make our chromosomes converge to optimal answer.

- Crossover makes better chromosomes by combining good chromosomes but mutation changes a chromosome directly and helps us escape a local maxima. We can probably reach the solution by only using one of them but it will be most likely not efficient for example uniform crossover may reach the answer by too many generations and in general only using mutation won't lead us to an optimal answer because it doesn't have the effect of crossover.

- Choosing the right amount for all constant values can accelerate our algorithm.

- Despite the crossover, mutation can solve this problem for us. Also we can use multiple starts to increase the probability of finding the actual answer instead of local maxima.

- We can define a maximum value for the number of the generations to stop the algorithm. also we can calculate variances of best chromosomes of some recent generations and compare it with a threshold and terminate if it is less than threshold.
